In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numbers
from itertools import combinations
from typing import Dict, List, Union

from numpy import exp, sqrt, pi
from pint import Quantity, UnitRegistry

# Define units
ureg = UnitRegistry()
mmHg = ureg.mmHg
Pa = ureg.Pa
m = ureg.m
mm = ureg.mm
um = ureg.um
nm = ureg.nm
s = ureg.s
minute = ureg.min
mL = ureg.mL


# Dictionary defining various subsets of the compartments and interfaces which 
# either share a parameter, or use the same functions to compute parameters.
SHARED_PARAMETERS = {
    "all": [
        "ecs",
        "pvs_arteries",
        "pvs_capillaries",
        "pvs_veins",
        "arteries",
        "capillaries",
        "veins",
    ],
    "pvs": ["pvs_arteries", "pvs_capillaries", "pvs_veins"],
    "csf": ["ecs", "pvs_arteries", "pvs_capillaries", "pvs_veins"],
    "blood": ["arteries", "capillaries", "veins"],
    "large_vessels": ["arteries", "veins"],
    "bbb": [
        ("pvs_arteries", "arteries"),
        ("pvs_capillaries", "capillaries"),
        ("pvs_veins", "veins"),
    ],
    "aef": [("ecs", "pvs_arteries"), ("ecs", "pvs_capillaries"), ("ecs", "pvs_veins")],
    "membranes": [
        ("pvs_arteries", "arteries"),
        ("pvs_capillaries", "capillaries"),
        ("pvs_veins", "veins"),
        ("ecs", "pvs_arteries"),
        ("ecs", "pvs_capillaries"),
        ("ecs", "pvs_veins")
    ],
    "connected" : [
        ("pvs_arteries", "pvs_capillaries"),
        ("pvs_capillaries", "pvs_veins"),
        ("arteries", "capillaries"),
        ("capillaries", "veins"),
    ],
    "connected_blood": [
        ("arteries", "capillaries"),
        ("capillaries", "veins"),
    ],
    "connected_pvs": [
        ("pvs_arteries", "pvs_capillaries"),
        ("pvs_capillaries", "pvs_veins"),
    ]
}

# Dictionary containing parameters with values found in literature, or values for which
# we have just assumed some value. All other parameters should be derived from these.
BASE_PARAMETERS = {
    "brain_volume": 2450.0 * mm**3,
    "csf_volume_fraction": 0.12, # 
    "human_brain_volume": 1.0e6 * mm**3,
    "human_brain_surface_area": 1.750e2 * mm**2,
    "diffusion_coefficient_free": {
        "inulin": 2.98e-4 * mm**2 / s,
        "amyloid_beta": 1.8e-4 * mm**2 / s,
    },
    "pressure_boundary": {
        "arteries": 120.0 * mmHg,
        "veins": 7.0 * mmHg,
        "pvs_arteries": 4.74 * mmHg,
        "pvs_veins": 3.36 * mmHg,
        "ecs": 3.74 * mmHg,
    },
    "tortuosity": 1.7,
    "osmotic_pressure": {
        "blood": 20.0 * mmHg,
    },
    "osmotic_pressure_fraction": {
        "csf": 0.2,  # # Osm. press. computed as this constat * osmotic_pressure-blood
    },
    "osmotic_reflection": {
        "inulin": {
            "membranes": 0.2,
            "connected": 0.0,
        }
    },
    "porosity": {
        "ecs": 0.14,
    },
    "vasculature_volume_fraction": 0.0329,
    "vasculature_fraction": {
        "arteries": 0.2,
        "capillaries": 0.1,
        "veins": 0.7,
    },
    "pvs_volume_fraction": 0.003,
    "viscosity": {
        "blood": 2.67e-3 * Pa * s,
        "csf": 7.0e-4 * Pa * s,
    },
    "permeability": {"ecs": 2.0e-11 * mm**2},
    "hydraulic_conductivity": {
        ("ecs", "arteries"): 9.1e-10 * mm / (Pa * s),
        ("ecs", "capillaries"): 1.0e-10 * mm / (Pa * s),
        ("ecs", "veins"): 2.0e-11 * mm / (Pa * s),
    },
    "surface_volume_ratio": {
        ("ecs", "arteries"): 3.0 / mm,
        ("ecs", "capillaries"): 9.0 / mm,
        ("ecs", "veins"): 3.0 / mm,
    },
    "flowrate": {
        "blood": 2.4 * mL / minute,
#         "blood": 700 * mL / minute, # human?
        "csf": 3.33 * mm**3 / s,
    },
    "pressure_drop": {
        ("arteries", "capillaries"): 50.0 * mmHg,
        ("capillaries", "veins"): 10.0 * mmHg,
        ("pvs_arteries", "pvs_capillaries"): 1.0 * mmHg,
        ("pvs_capillaries", "pvs_veins"): 0.25 * mmHg,
        ("c_prox", "c_dist"): 60.0 * mmHg,
    },
    "resistance": {
        "ecs": 4.56 * Pa * s / mm**3,
        "pvs_arteries": 1.14 * mmHg / (mL / minute),
        "pvs_capillaries": 32.4 * mmHg / (mL / minute),
        "pvs_veins": 1.75e-3 * mmHg / (mL / minute),
    },
    "resistance_interface": {
        ("ecs", "arteries"): 0.57 * mmHg / (mL / minute),
        ("ecs", "capillaries"): 125.31 * mmHg / (mL / minute),  # FIXME: Wrong value
        ("ecs", "veins"): 0.64 * mmHg / (mL / minute),
    },
    "diameter": {
        "arteries": 50.0 * um,
        "capillaries": 10.0 * um,
        "veins": 50.0 * um,
    },
    "solute_radius": {
        "inulin": 15.2e-7 * mm,  # Sauce?
        "amyloid_beta": 0.9 * nm,
    },
    # Related to permeability of BBB. Since this work is restricted to inulin, only AEF is of interest.
    "membranes": {
        "layertype": {
            "glycocalyx": "fiber",
            "inner_endothelial_cleft": "pore",
            "endothelial_junction": "pore",
            "outer_endothelial_cleft": "pore",
            "basement_membrane": "fiber",
            "aef": "pore",
        },
        "thickness": {
            "glycocalyx": {
                "arteries": 400.0 * nm,
                "capillaries": 250.0 * nm,
                "veins": 100.0 * nm,
            },
            "inner_endothelial_cleft": 350.0 * nm,
            "endothelial_junction": 11.0 * mm,
            "outer_endothelial_cleft": 339.0 * nm,  # Total endothelial length 700nm
            "basement_membrane": {
                "arteries": 80.0 * nm,
                "capillaries": 30.0 * nm,
                "veins": 20.0 * nm,
            },
            "aef": 1000.0 * nm,
        },
        "elementary_radius": {
            "glycocalyx": 6.0 * nm,
            "inner_endothelial_cleft": 9.0 * nm,
            "endothelial_junction": {
                "arteries": 0.5 * nm,
                "capillaries": 2.5 * nm,
                "veins": 10.0 * nm,
            },
            "outer_endothelial_cleft": 9.0 * nm,
            "basement_membrane": {
                "arteries": 80.0 * nm,
                "capillaries": 30.0 * nm,
                "veins": 20.0 * nm,
            },
            "aef": {
                "arteries": 250.0 * nm,
                "capillaries": 10.0 * nm,
                "veins": 250.0 * nm,
            },
        },
        "fiber_volume_fraction": {"glycocalyx": 0.326, "basement_membrane": 0.5},
    },
}

In [3]:
def get_base_parameters():
    """Return a copy of the BASE_PARAMETERS, containing parameter values explicitly found in literature.
    Based on these values remaining paramters will be computed."""
    return {**BASE_PARAMETERS}


def get_shared_parameters():
    """Return a copy of the SHARED_PARAMETERS, defining various subsets of compartments and interfaces."""
    return {**SHARED_PARAMETERS}


def pvs(v):
    return f"pvs_{v}"


def isquantity(x):
    return isinstance(x, Quantity) or isinstance(x, numbers.Complex)


def print_quantities(p, offset, depth=0):
    """Pretty printing of dictionaries filled with pint.Quantities"""
    format_size = offset - depth * 2
    for key, value in p.items():
        if isinstance(value, dict):
            print(f"{depth*'  '}{str(key)}")
            print_quantities(value, offset, depth=depth + 1)
        else:
            if isquantity(value):
                print(f"{depth*'  '}{str(key):<{format_size+1}}: {value:.3e}")
            else:
                print(f"{depth*'  '}{str(key):<{format_size+1}}: {value}")
                
                
def distribute_subset_parameters(base, subsets):
    """Take any parameter entry indexed by the name of some subset (e.g. 'blood'), 
    and create a new entry for each of the compartments/interfaces included in the
    given subset."""
    extended = {}
    for param_name, param_value in base.items():
        if not isinstance(param_value, dict):
            extended[param_name] = param_value
        else:
            param_dict = {**param_value} # Copy old values

            # Check if any of the entries refer to a subset of compartments...
            for idx, val in param_value.items():
                if idx in subsets:
                    # ... and insert one entry for each of the compartment in the subset.
                    for compartment in subsets[idx]:
                        param_dict[compartment] = val
            extended[param_name] = param_dict
    return extended


def make_dimless(params, param_units):
    """Converts all quantities to the units specified by
    param_units, before converting it to a dimless number."""
    dimless = {}
    for key, val in params.items():
        if isinstance(val, dict):
            dimless[key] = make_dimless(val, param_units[key])
        elif isinstance(val, Quantity):
            dimless[key] = val.to(param_units).magnitude
        else:
            dimless[key] = val
    return dimless


def get_interface_parameter(param, compartments, is_symmetric=False):
    """Create a dictionary with entries for each permutation of 
    two compartments. Entries which are not present will either be
    set to zero, or to the mirrored value if the parameter is symmetric."""
    out = {}
    for i, j in permutations(compartments, 2):
        if (i, j) in param:
            out[(i, j)] = param[(i, j)]
        elif (j, i) in param and is_symmetric:
            out[(i, j)] = param[(j, i)]
        else:
            out[(i, j)] = out[(i, j)] = 0.0
    return out

### Effective Diffusion Coefficient

In [4]:
base = get_base_parameters()
base = distribute_subset_parameters(base, SHARED_PARAMETERS)

In [5]:
## DIFFUSION COEFFICIENTS
def get_effective_diffusion(params, solute):
    Dfree = params["diffusion_coefficient_free"][solute]
    tortuosity = params["tortuosity"]
    return {key: Dfree / tortuosity**2 for key in SHARED_PARAMETERS['all']}


print("= Diffusion Coefficient Effective =")
phi = get_effective_diffusion(base, "inulin")
print_quantities(phi, 40)
print()

= Diffusion Coefficient Effective =
ecs                                      : 1.031e-04 millimeter ** 2 / second
pvs_arteries                             : 1.031e-04 millimeter ** 2 / second
pvs_capillaries                          : 1.031e-04 millimeter ** 2 / second
pvs_veins                                : 1.031e-04 millimeter ** 2 / second
arteries                                 : 1.031e-04 millimeter ** 2 / second
capillaries                              : 1.031e-04 millimeter ** 2 / second
veins                                    : 1.031e-04 millimeter ** 2 / second



In [6]:
# POROSITIES
def get_porosities(params):
    phi = {"ecs" : params["porosity"]["ecs"]}
    phi_B = params["vasculature_volume_fraction"]
    phi_PV = params["pvs_volume_fraction"]
    for vi in SHARED_PARAMETERS["blood"]:
        fraction_vi = params["vasculature_fraction"][vi]
        phi[vi] = fraction_vi * phi_B
        phi[pvs(vi)] = fraction_vi * phi_PV
    return phi

print("= Porosities =")
phi = get_porosities(base)
print_quantities(phi, 40)
print()

= Porosities =
ecs                                      : 1.400e-01
arteries                                 : 6.580e-03
pvs_arteries                             : 6.000e-04
capillaries                              : 3.290e-03
pvs_capillaries                          : 3.000e-04
veins                                    : 2.303e-02
pvs_veins                                : 2.100e-03



### Permeability 

#### Viscosity 

In [7]:
def get_viscosities(params):
    viscosities = params["viscosity"]
    return viscosities


print("= Viscosities =")
k = get_viscosities(base)
print_quantities(base["viscosity"], 40)
print()

= Viscosities =
blood                                    : 2.670e-03 pascal * second
csf                                      : 7.000e-04 pascal * second
arteries                                 : 2.670e-03 pascal * second
capillaries                              : 2.670e-03 pascal * second
veins                                    : 2.670e-03 pascal * second
ecs                                      : 7.000e-04 pascal * second
pvs_arteries                             : 7.000e-04 pascal * second
pvs_capillaries                          : 7.000e-04 pascal * second
pvs_veins                                : 7.000e-04 pascal * second



#### Resistances

In [8]:
def get_resistances(params):
    R = {**params["resistance"], **blood_resistance(params)}
    return R

    
def blood_resistance(params):
    Q = params["flowrate"]["blood"]
    dp = params["pressure_drop"]
    return {
        "arteries": dp[("arteries", "capillaries")] / Q,
        "capillaries": dp[("c_prox", "c_dist")] / Q,
        "veins": dp[("capillaries", "veins")] / Q,
    }

print("= Resistances =")
R = get_resistances(base)
print_quantities(R, 40)
print()

= Resistances =
ecs                                      : 4.560e+00 pascal * second / millimeter ** 3
pvs_arteries                             : 1.140e+00 millimeter_Hg * minute / milliliter
pvs_capillaries                          : 3.240e+01 millimeter_Hg * minute / milliliter
pvs_veins                                : 1.750e-03 millimeter_Hg * minute / milliliter
arteries                                 : 2.083e+01 millimeter_Hg * minute / milliliter
capillaries                              : 2.500e+01 millimeter_Hg * minute / milliliter
veins                                    : 4.167e+00 millimeter_Hg * minute / milliliter



#### Permeabilities

In [9]:
# PERMEABILITIES
def get_permeabilities(p):
    R = get_resistances(p)
    mu = p["viscosity"]
    k = {'ecs': p["permeability"]["ecs"]}
    length_area_ratio = R["ecs"] * k["ecs"] / mu["ecs"]
    for comp in [*SHARED_PARAMETERS["pvs"], *SHARED_PARAMETERS["blood"]]:
        k[comp] = length_area_ratio * mu[comp] / R[comp]

    return {key: val.to("mm^2") for key, val in k.items()}


print("= Permeabilities =")
k = get_permeabilities(base)
print_quantities(k, 40)
print()

= Permeabilities =
ecs                                      : 2.000e-11 millimeter ** 2
pvs_arteries                             : 1.000e-11 millimeter ** 2
pvs_capillaries                          : 3.519e-13 millimeter ** 2
pvs_veins                                : 6.515e-09 millimeter ** 2
arteries                                 : 2.087e-12 millimeter ** 2
capillaries                              : 1.739e-12 millimeter ** 2
veins                                    : 1.044e-11 millimeter ** 2



## Fluid Transfer Coefficients
### Convective Fluid Transfer

In [10]:
def get_convective_fluid_transfer(params):
    T = {}
    V = params["human_brain_volume"]
    # Compute membrane transfer coefficients.
    # FIXME: Apparent error in resistance ecs-capillaries.
    for vi in SHARED_PARAMETERS["blood"]:
        L_ecs_vi = params["hydraulic_conductivity"][("ecs", vi)]
        surface_ratio = params["surface_volume_ratio"][("ecs", vi)]
        T_ecs_vi = L_ecs_vi * surface_ratio
        R_ecs_vi = params["resistance_interface"][("ecs", vi)]

        T[("ecs", pvs(vi))] = 1.0 / (V * R_ecs_vi)
        T[(pvs(vi), vi)] = compute_partial_fluid_transfer(V, R_ecs_vi, T_ecs_vi)

    # Compute connected transfer coefficients.
    for vi, vj in SHARED_PARAMETERS["connected_blood"]:
        Q = params["flowrate"]
        dp = params["pressure_drop"]
        T[(vi, vj)] = compute_connected_fluid_transfer(V, Q["blood"], dp[(vi, vj)])
        T[(pvs(vi), pvs(vj))] = compute_connected_fluid_transfer(
            V, Q["csf"], dp[(pvs(vi), pvs(vj))]
        )
    return {key: val.to(1 / (Pa * s)) for key, val in T.items()}


def compute_partial_fluid_transfer(brain_volume, resistance, total_transfer):
    new_resistance = 1.0 / (total_transfer * brain_volume) - resistance
    return 1.0 / (new_resistance * brain_volume)


def compute_connected_fluid_transfer(brain_volume, flow_rate, pressure_drop):
    return flow_rate / (pressure_drop * brain_volume)


print("= Convective Fluid Transfer =")
gamma = get_convective_fluid_transfer(base)
print_quantities(gamma, 40)
print()

= Convective Fluid Transfer =
('ecs', 'pvs_arteries')                  : 2.193e-07 / pascal / second
('pvs_arteries', 'arteries')             : 2.764e-09 / pascal / second
('ecs', 'pvs_capillaries')               : 9.976e-10 / pascal / second
('pvs_capillaries', 'capillaries')       : 9.198e-09 / pascal / second
('ecs', 'pvs_veins')                     : 1.953e-07 / pascal / second
('pvs_veins', 'veins')                   : 6.002e-11 / pascal / second
('arteries', 'capillaries')              : 6.000e-09 / pascal / second
('pvs_arteries', 'pvs_capillaries')      : 2.498e-08 / pascal / second
('capillaries', 'veins')                 : 3.000e-08 / pascal / second
('pvs_capillaries', 'pvs_veins')         : 9.991e-08 / pascal / second



### Osmotic Transfer Coeffieccients

In [11]:
def get_osmotic_pressure(params):
    pi_B = params["osmotic_pressure"]["blood"]
    csf_factor = params["osmotic_pressure_fraction"]["csf"]
    
    pi = {}
    for x in SHARED_PARAMETERS["all"]:
        if x in SHARED_PARAMETERS["blood"]:
            pi[x] = pi_B
        elif x in SHARED_PARAMETERS["csf"]:
            pi[x] = csf_factor * pi_B

    return pi


def get_osmotic_reflection(params, solute):
    sigma = {}
    for interface in SHARED_PARAMETERS["membranes"]:
        sigma[interface] = params["osmotic_reflection"][solute]["membranes"]
    
    for interface in SHARED_PARAMETERS["connected"]:
        sigma[interface] = params["osmotic_reflection"][solute]["connected"] # TODO: Verify correctness here as well.
    return sigma

print("= Osmotic Fluid Transfer =")
osmotic = {
    "osmotic_reflection": get_osmotic_reflection(base, "inulin"),
    "osmotic_pressure": get_osmotic_pressure(base)
}
print_quantities(osmotic, 40)
print()

= Osmotic Fluid Transfer =
osmotic_reflection
  ('pvs_arteries', 'arteries')           : 2.000e-01
  ('pvs_capillaries', 'capillaries')     : 2.000e-01
  ('pvs_veins', 'veins')                 : 2.000e-01
  ('ecs', 'pvs_arteries')                : 2.000e-01
  ('ecs', 'pvs_capillaries')             : 2.000e-01
  ('ecs', 'pvs_veins')                   : 2.000e-01
  ('pvs_arteries', 'pvs_capillaries')    : 0.000e+00
  ('pvs_capillaries', 'pvs_veins')       : 0.000e+00
  ('arteries', 'capillaries')            : 0.000e+00
  ('capillaries', 'veins')               : 0.000e+00
osmotic_pressure
  ecs                                    : 4.000e+00 millimeter_Hg
  pvs_arteries                           : 4.000e+00 millimeter_Hg
  pvs_capillaries                        : 4.000e+00 millimeter_Hg
  pvs_veins                              : 4.000e+00 millimeter_Hg
  arteries                               : 2.000e+01 millimeter_Hg
  capillaries                            : 2.000e+01 millimeter_Hg
  vei

## Solute Transfer Coefficient
### Convective Solute Transfer

In [12]:
def get_convective_solute_transfer(params, solute):
    sigma = get_osmotic_reflection(params, solute)
    G = get_convective_fluid_transfer(params)
    return {ij: G[ij] * ( 1 - sigma[ij]) for ij in G}

print("= Convective Solute Transfer =")
gamma_tilde = get_convective_solute_transfer(base, "inulin")
print_quantities(gamma_tilde, 40)
print()

= Convective Solute Transfer =
('ecs', 'pvs_arteries')                  : 1.755e-07 / pascal / second
('pvs_arteries', 'arteries')             : 2.212e-09 / pascal / second
('ecs', 'pvs_capillaries')               : 7.981e-10 / pascal / second
('pvs_capillaries', 'capillaries')       : 7.359e-09 / pascal / second
('ecs', 'pvs_veins')                     : 1.563e-07 / pascal / second
('pvs_veins', 'veins')                   : 4.801e-11 / pascal / second
('arteries', 'capillaries')              : 6.000e-09 / pascal / second
('pvs_arteries', 'pvs_capillaries')      : 2.498e-08 / pascal / second
('capillaries', 'veins')                 : 3.000e-08 / pascal / second
('pvs_capillaries', 'pvs_veins')         : 9.991e-08 / pascal / second



### Diffusive Solute Transfer

#### Interface Permeabilities

In [13]:
def diffusive_permeabilities_inulin(params):
    P = {}  
    # Permeability over membranes.
    for vi in SHARED_PARAMETERS["blood"]:
        d_vi = params['diameter'][vi]
        R_aef_vi = diffusive_resistance_aef_inulin(params, vi)
        P[(pvs(vi), vi)] = 0.0 * mm / s
        P[("ecs", pvs(vi))] = 1.0 / (pi * d_vi * R_aef_vi)

    # Assume purely convection-driven transport between connected compartments.
    for i, j in SHARED_PARAMETERS["connected"]:
        P[(i, j)] = 0.0 * mm / s
        
    return {key: val.to('mm / s') for key, val in P.items()}

def diffusive_resistance_aef_inulin(params, vessel):
    D_free = params["diffusion_coefficient_free"]["inulin"]
    membranes = params["membranes"]

    
    thickness = membranes["thickness"]["aef"]
    B_aef = membranes["elementary_radius"]["aef"][vessel]
    solute_radius = params["solute_radius"]["inulin"]
    D_eff = diffusion_porous(D_free, solute_radius, B_aef)
    return resistance_aef_inulin(thickness, B_aef, D_eff)


def diffusion_porous(
    D_free: Quantity, solute_radius: Quantity, pore_radius: Quantity
) -> Quantity:
    beta = solute_radius / pore_radius
    return D_free * (
        1.0
        - 2.10444 * beta  # **6
        + 2.08877 * beta**3
        - 0.094813 * beta**5
        - 1.372 * beta**6
    )


def resistance_aef_inulin(layer_thickness, pore_radius, effective_diffusion):
    return layer_thickness / (2.0 * pore_radius * effective_diffusion)

print("= Diffusive Permeabilities of Interfaces (Transfer Coefficients Below)=")
P = diffusive_permeabilities_inulin(base)
print_quantities(P, 40)
print()

= Diffusive Permeabilities of Interfaces (Transfer Coefficients Below)=
('pvs_arteries', 'arteries')             : 0.000e+00 millimeter / second
('ecs', 'pvs_arteries')                  : 9.364e-04 millimeter / second
('pvs_capillaries', 'capillaries')       : 0.000e+00 millimeter / second
('ecs', 'pvs_capillaries')               : 1.304e-04 millimeter / second
('pvs_veins', 'veins')                   : 0.000e+00 millimeter / second
('ecs', 'pvs_veins')                     : 9.364e-04 millimeter / second
('pvs_arteries', 'pvs_capillaries')      : 0.000e+00 millimeter / second
('pvs_capillaries', 'pvs_veins')         : 0.000e+00 millimeter / second
('arteries', 'capillaries')              : 0.000e+00 millimeter / second
('capillaries', 'veins')                 : 0.000e+00 millimeter / second



#### Transfer Coefficients

In [14]:
def get_diffusive_solute_transfer_inulin(params):
    P = diffusive_permeabilities_inulin(params)
    surf_volume_ratio = params["surface_volume_ratio"]
    L = {}
    for vi in SHARED_PARAMETERS["blood"]:
        L[('ecs', pvs(vi))] = P[('ecs', pvs(vi))] * surf_volume_ratio[('ecs', vi)]
        L[(pvs(vi), vi)] = P[(pvs(vi), vi)] * surf_volume_ratio[('ecs', vi)]
    return {key: val.to(1 / (s)) for key, val in L.items()}

print("= Diffusive Transfer Coefficients =")
lambda_ij = get_diffusive_solute_transfer_inulin(base)
print_quantities(lambda_ij, 40)
print()

= Diffusive Transfer Coefficients =
('ecs', 'pvs_arteries')                  : 2.809e-03 / second
('pvs_arteries', 'arteries')             : 0.000e+00 / second
('ecs', 'pvs_capillaries')               : 1.174e-03 / second
('pvs_capillaries', 'capillaries')       : 0.000e+00 / second
('ecs', 'pvs_veins')                     : 2.809e-03 / second
('pvs_veins', 'veins')                   : 0.000e+00 / second



## Boundary Parameters 

### Pressure Boundary

In [15]:
def get_boundary_hydraulic_permeabilities(p):
    # FIXME: This function is seemingly wrong.
    R_bdry = {
        'ecs': 2 * p['resistance']['pvs_arteries'],
        'pvs_arteries': p['resistance']['pvs_arteries'],
        'arteries': blood_resistance(p)['arteries'],
    }
    L_bdry = {}
    S = p["human_brain_surface_area"]
    for j in R_bdry:
        L_bdry[j] = 1.0 / (R_bdry[j] * S)
    return {key: value.to("mm / (Pa * s)") for key, value in L_bdry.items()}

print("= Pressure Boundary Conditions =")
p_bdry = {
    "L_boundary": get_boundary_hydraulic_permeabilities(base),
    "pressure_boundary": base["pressure_boundary"]
}
print_quantities(p_bdry, 20)

= Pressure Boundary Conditions =
L_boundary
  ecs                : 3.133e-04 millimeter / pascal / second
  pvs_arteries       : 6.266e-04 millimeter / pascal / second
  arteries           : 3.429e-05 millimeter / pascal / second
pressure_boundary
  arteries           : 1.200e+02 millimeter_Hg
  veins              : 7.000e+00 millimeter_Hg
  pvs_arteries       : 4.740e+00 millimeter_Hg
  pvs_veins          : 3.360e+00 millimeter_Hg
  ecs                : 3.740e+00 millimeter_Hg


## List All Parameters
("all" as in every parameter that is present in the model equation, not the intermediate parameters)

In [16]:
def compute_parameters(params):
    return {
        "permeability": get_permeabilities(params),
        "viscosity": params["viscosity"],
        "porosity": get_porosities(params),
        "convective_fluid_transfer": get_convective_fluid_transfer(params),
        "osmotic_pressure": get_osmotic_pressure(params),
        "osmotic_reflection": get_osmotic_reflection(params, "inulin"),
        "effective_diffusion": get_effective_diffusion(params, "inulin"),
        "diffusive_solute_transfer": get_diffusive_solute_transfer_inulin(params),
        "convective_solute_transfer": get_convective_solute_transfer(params, "inulin"),
        "hydarulic_conductivity_boundaries": get_boundary_hydraulic_permeabilities(base),
        "pressure_boundaries": base["pressure_boundary"]
    }

In [17]:
params = compute_parameters(base)
print_quantities(params, 40)

permeability
  ecs                                    : 2.000e-11 millimeter ** 2
  pvs_arteries                           : 1.000e-11 millimeter ** 2
  pvs_capillaries                        : 3.519e-13 millimeter ** 2
  pvs_veins                              : 6.515e-09 millimeter ** 2
  arteries                               : 2.087e-12 millimeter ** 2
  capillaries                            : 1.739e-12 millimeter ** 2
  veins                                  : 1.044e-11 millimeter ** 2
viscosity
  blood                                  : 2.670e-03 pascal * second
  csf                                    : 7.000e-04 pascal * second
  arteries                               : 2.670e-03 pascal * second
  capillaries                            : 2.670e-03 pascal * second
  veins                                  : 2.670e-03 pascal * second
  ecs                                    : 7.000e-04 pascal * second
  pvs_arteries                           : 7.000e-04 pascal * second
  pvs_capil

## General Diffusive Permeabilities (for e.g. amyloid beta)